In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

sys.path.append("../")
from pyfiles.processsound import trim_audio_and_save

import json
sys.path.append("./../../hifi-gan/")
from meldataset import mel_spectrogram, load_wav
from env import AttrDict, build_env
config = f'../../hifi-gan/config_16kHz.json'
with open(config) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [17]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/PD-AST/SLT/"
# dataset_dir = data_dir + "Dataset/PD-AST/SLT_add/"
fs = 16000
trim_threshold_in_db = 30
# speakers = ["English", "Korean"]
speakers = ["Hindi", "English"]

def get_mel_spectrogram(path, MAX_WAV_VALUE=32768.0):
    audio, _ = load_wav(path, h.sampling_rate)
    if audio.max()>1.0:
        audio = audio / MAX_WAV_VALUE
    audio = torch.FloatTensor(audio)
    audio = audio.unsqueeze(0)
    mel = mel_spectrogram(audio, h.n_fft, h.num_mels,
                          h.sampling_rate, h.hop_size, h.win_size, h.fmin, h.fmax,
                          center=False)
    mel = np.array(mel[0])
    return mel

In [18]:
save = True
tempfile = "temp1.wav"

for spk in speakers:
    print(spk)
    mel_dir = f"{dataset_dir}{spk}/hifiganmel/"
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
        mel = get_mel_spectrogram(tempfile)
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

Hindi


100%|██████████| 1132/1132 [02:16<00:00,  8.27it/s]


English


100%|██████████| 1132/1132 [01:56<00:00,  9.72it/s]


In [19]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
# dataset_dir = data_dir + "Dataset/PD-AST/SLT/"
dataset_dir = data_dir + "Dataset/PD-AST/SLT_add/"
fs = 16000
trim_threshold_in_db = 30
# speakers = ["English", "Korean"]
speakers = ["Hindi", "English"]

def get_mel_spectrogram(path, MAX_WAV_VALUE=32768.0):
    audio, _ = load_wav(path, h.sampling_rate)
    if audio.max()>1.0:
        audio = audio / MAX_WAV_VALUE
    audio = torch.FloatTensor(audio)
    audio = audio.unsqueeze(0)
    mel = mel_spectrogram(audio, h.n_fft, h.num_mels,
                          h.sampling_rate, h.hop_size, h.win_size, h.fmin, h.fmax,
                          center=False)
    mel = np.array(mel[0])
    return mel

In [20]:
save = True
tempfile = "temp1.wav"

for spk in speakers:
    print(spk)
    mel_dir = f"{dataset_dir}{spk}/hifiganmel/"
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
        mel = get_mel_spectrogram(tempfile)
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

Hindi


100%|██████████| 4500/4500 [05:47<00:00, 12.93it/s]


English


100%|██████████| 4500/4500 [05:34<00:00, 13.44it/s]
